In [15]:
from dotenv import load_dotenv
import os
import sys
import logging

from core.data_sources import CLOBDataSource
from core.services.backend_api_client import BackendAPIClient

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

logging.basicConfig(level=logging.INFO)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

load_dotenv()

from core.services.mongodb_client import MongoDBClient

mongo_client = MongoDBClient(**{
        "username": os.getenv('MONGO_INITDB_ROOT_USERNAME', "admin"),
        "password": os.getenv('MONGO_INITDB_ROOT_PASSWORD', "admin"),
        "host": os.getenv('MONGO_HOST', 'localhost'),
        "port": os.getenv('MONGO_PORT', 27017),
        "database": "mongodb"
    })
await mongo_client.connect()
clob = CLOBDataSource()

In [20]:
import plotly.express as px
import pandas as pd

# TODO: Add below line
# df['cond'] = np.where(((df['quote'] == df['pair1']) & (df['rate_difference'] > 0)) | ((df['quote'] == df['pair2']) & (df['rate_difference'] < 0)), 1, 0)

controller_configs_data = await mongo_client.get_controller_config_data()
extra_info = [controller_config_data["extra_info"] for controller_config_data in controller_configs_data]
extra_info_df = pd.DataFrame(extra_info)

fig = px.scatter(
    extra_info_df,
    x="coint_value",
    y="rate_difference",
    custom_data=[extra_info_df.index, extra_info_df["base_rate"], extra_info_df["quote_rate"]]  # Pass specific columns
)

# Customize the hover template to include additional information
fig.update_traces(
    hovertemplate="""
    <b>Index:</b> %{customdata[0]}<br>
    <b>Base Rate:</b> %{customdata[1]}<br>
    <b>Quote Rate:</b> %{customdata[2]}<br>
    <extra></extra>
    """
)

fig.show()


In [21]:
controller_config_data = controller_configs_data[52]

update_candles = False
days_to_download = 14
interval = "15m"

extra_info = controller_config_data["extra_info"]
controller_config = controller_config_data["config"]
if update_candles:
        base_candles = clob.get_candles_last_days(connector_name=controller_config["connector_name"],
                                                  trading_pair=controller_config["base_trading_pair"],
                                                  interval=interval,
                                                  days=days_to_download)
        quote_candles = clob.get_candles_last_days(connector_name=controller_config["connector_name"],
                                                  trading_pair=controller_config["quote_trading_pair"],
                                                  interval=interval,
                                                  days=days_to_download)
else:
        clob.load_candles_cache('../..')
        base_candles = clob.candles_cache[(controller_config["connector_name"],
                                           controller_config["base_trading_pair"],
                                           interval)]
        quote_candles = clob.candles_cache[(controller_config["connector_name"],
                                           controller_config["quote_trading_pair"],
                                           interval)]


In [22]:
controller_config

{'id': 'SOLVUSDT_GOATUSDT_coint_config',
 'controller_name': 'stat_arb',
 'controller_type': 'generic',
 'total_amount_quote': 1000,
 'manual_kill_switch': None,
 'connector_name': 'binance_perpetual',
 'base_trading_pair': 'SOLV-USDT',
 'quote_trading_pair': 'GOAT-USDT',
 'base_side': 1,
 'grid_config_base': {'end_price': nan,
  'limit_price': 0.0,
  'start_price': nan,
  'order_frequency': 5,
  'min_order_amount_quote': 5},
 'grid_config_quote': {'end_price': nan,
  'limit_price': 0.0,
  'start_price': nan,
  'order_frequency': 5,
  'min_order_amount_quote': 5},
 'leverage': 50,
 'position_mode': 'HEDGE',
 'min_spread_between_orders': 0.0002,
 'max_open_orders': 3,
 'max_orders_per_batch': 1,
 'activation_bounds': 0.0003,
 'safe_extra_spread': 0.0002,
 'deduct_base_fees': False,
 'triple_barrier_config': {'open_order_type': 3,
  'stop_loss': 0.1,
  'stop_loss_order_type': 1,
  'take_profit': 0.0008,
  'take_profit_order_type': 3,
  'time_limit': 259200,
  'time_limit_order_type': 1,


In [23]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create the figure
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles=[controller_config["base_trading_pair"], controller_config["quote_trading_pair"]],
    x_title="Time",
    y_title="Price"
)

# Add base market candlesticks
fig.add_trace(
    go.Candlestick(
        x=base_candles.data.index,
        open=base_candles.data.open,
        high=base_candles.data.high,
        low=base_candles.data.low,
        close=base_candles.data.close
    ),
    row=1, col=1
)

# Add quote market candlesticks
fig.add_trace(
    go.Candlestick(
        x=quote_candles.data.index,
        open=quote_candles.data.open,
        high=quote_candles.data.high,
        low=quote_candles.data.low,
        close=quote_candles.data.close
    ),
    row=2, col=1
)

# Add horizontal lines for the base market
fig.add_hline(
    y=controller_config["grid_config_base"]["start_price"],
    row=1, col=1,
    line=dict(color="green", width=4)  # Double sized
)
fig.add_hline(
    y=controller_config["grid_config_base"]["end_price"],
    row=1, col=1,
    line=dict(color="green", width=4)  # Double sized
)
fig.add_hline(
    y=controller_config["grid_config_base"]["limit_price"],
    row=1, col=1,
    line=dict(color="green", dash="dash")  # Dashed
)

# Add horizontal lines for the quote market
fig.add_hline(
    y=controller_config["grid_config_quote"]["start_price"],
    row=2, col=1,
    line=dict(color="red", width=4)  # Double sized
)
fig.add_hline(
    y=controller_config["grid_config_quote"]["end_price"],
    row=2, col=1,
    line=dict(color="red", width=4)  # Double sized
)
fig.add_hline(
    y=controller_config["grid_config_quote"]["limit_price"],
    row=2, col=1,
    line=dict(color="red", dash="dash")  # Dashed
)

# Update layout
fig.update_layout(
    template="plotly_dark",
    # xaxis_rangeslider_visible=False,  # This controls the rangeslider visibility for x-axis
    xaxis2_rangeslider_visible=False,  # Disable for the second subplot
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent background
    paper_bgcolor='rgba(0, 0, 0, 0.1)',  # Lighter shade for the paper
    font={"color": 'white', "size": 12},  # Consistent font color and size
    height=1000,
    hovermode="x unified",
    showlegend=False
)

# Show the figure
print(f"Coint Value: {extra_info['coint_value']:.3f} || Rate Difference (Base - Quote):  {extra_info['rate_difference']:.5f}% || Base Beta {extra_info['base_beta']:.3f} || Quote Beta: {extra_info['quote_beta']:.3f} || Base Rate: {extra_info['base_rate']} || Quote Rate: {extra_info['quote_rate']}")
fig.show()


Coint Value: 0.002 || Rate Difference (Base - Quote):  -0.00010% || Base Beta 0.866 || Quote Beta: 1.062 || Base Rate: 5e-05 || Quote Rate: 0.00014572


In [7]:
controller_config["total_amount_quote"] = 1000.0
controller_config["leverage"] = 50
controller_config["'min_spread_between_orders'"] = 0.0002
controller_config["triple_barrier_config"] = {
    'open_order_type': 3,
    'stop_loss': 0.1,
    'stop_loss_order_type': 1,
    'take_profit': 0.0008,
    'take_profit_order_type': 3,
    'time_limit': 259200,
    'time_limit_order_type': 1,
    'trailing_stop': {'activation_price': 0.03, 'trailing_delta': 0.005}
}
controller_configs = [controller_config]

In [6]:
backend_api = BackendAPIClient(host=os.getenv("TRADING_HOST", "localhost"),
                               username=os.getenv("BACKEND_API_USER", "admin"),
                               password=os.getenv("BACKEND_API_PASSWORD", "admin"))
await backend_api.add_controller_config(controller_config)

{'message': 'Controller configuration uploaded successfully.'}

In [ ]:
from datetime import datetime

await backend_api.deploy_script_with_controllers(bot_name=f"{datetime.strftime(datetime.now(), '%Y-%m-%d_%H:%M:%S')}_cointegration",
                                                 controller_configs=controller_configs,
                                                 script_name="v2_with_controllers.py",
                                                 image_name="hummingbot/hummingbot:latest",
                                                 credentials="master_account",
                                                 time_to_cash_out=60 * 60 * 24 * 3,
                                                 max_global_drawdown=0.5,
                                                 max_controller_drawdown=0.5)

## TODO: setup new task config to run the three db feeds (funding rate, cointegration and config generation)